In [0]:
import numpy as np
import re
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
import textdistance
from string import punctuation
punct = set(punctuation)

In [5]:
from google.colab import drive                  #настройка доступа к файлам на Google drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
corpus = [sent.split() for sent in open('/content/gdrive/My Drive/corpus_ng.txt', encoding='utf8').read().splitlines()] 
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [0]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [0]:
def get_closest_hybrid_match(text, X, vec, metric=textdistance.levenshtein):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:5]
    variants =[id2word[top] for top in topn]
    
    similarities = Counter()
    for word in variants:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(1)[0][0]

In [9]:
get_closest_hybrid_match('солют', X, vec)

'салют'

In [0]:
bad = open('/content/gdrive/My Drive/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('/content/gdrive/My Drive/correct_sents.txt', encoding='utf8').read().splitlines()

In [0]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [12]:
correct = 0
total = 0
mistakes ={}

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    for pair in word_pairs:
        
        predicted = get_closest_hybrid_match(pair[1], X, vec)
        if predicted == pair[0]:
            correct += 1
        else:
            mistakes[pair]=predicted
        total += 1
    if not i % 10:
        print(i)
        print(correct/total)                                                     #оценка качества (данные для каждого, кратного 10, кол-ва пар)

0
0.5333333333333333
10
0.8076923076923077
20
0.8095238095238095
30
0.8106060606060606
40
0.8128654970760234
50
0.8164251207729468
60
0.8083333333333333
70
0.8118811881188119
80
0.8161530286928799
90
0.8172888015717092
100
0.8127240143369175
110
0.8161209068010076
120
0.8190255220417634
130
0.8175824175824176
140
0.8140806561859193
150
0.8225602027883396
160
0.8213443396226415
170
0.8220101066816395
180
0.8194444444444444
190
0.8211422845691383
200
0.823953823953824
210
0.8262250453720508
220
0.8269065981148244
230
0.8279260780287474
240
0.8326246617703904
250
0.82959970620639
260
0.8301090397467464
270
0.828262339418526
280
0.829945143594708
290
0.8276827371695179
300
0.8293269230769231
310
0.8296187683284457
320
0.8290331726679898
330
0.8282242474454571
340
0.8273168103448276
350
0.8267469248887725
360
0.8269133299543842
370
0.8280823616968495
380
0.8287092882991556
390
0.829227854097584
400
0.8302018093249826
410
0.8307033363390441
420
0.8304184193048484
430
0.8306069737408524
440
0

In [13]:
print(mistakes)

{('симпатичнейшее', 'симпатичнейшое'): 'пластичнейшими', ('шпионское', 'шпионское'): 'шпионские', ('гламурный', 'гламурный'): 'лагерный', ('бонда', 'бонда'): 'банд', ('superheadz', 'superheadz'): 'sueddeutsche', ('clap', 'clap'): 'place', ('camera', 'camera'): 'america', ('получатся', 'полчатся'): 'ополчатся', ('язычки', 'язычки'): 'язычка', ('очень', 'оччччень'): 'чечни', ('милые', 'милые'): 'милы', ('насчет', 'нащщот'): 'защищено', ('чавеса', 'чавеса'): 'чавес', ('попавшим', 'попавшим'): 'пропавшим', ('аварийно-спасательных', 'аварийно-спасательных'): 'аварийно-восстановительных', ('в', 'вобщем'): 'общем', ('общем', 'как'): 'как', ('как', 'вы'): 'вы', ('вы', 'знаете'): 'знаете', ('знаете', 'из'): 'из', ('из', 'моего'): 'моего', ('моего', 'не'): 'не', ('недавнего', 'давнего'): 'давнего', ('пропажу', 'пропажу'): 'продажу', ('почте.ру', 'почте.ру'): 'т.п', ('хорошо', 'хороше'): 'хорошее', ('рите', 'рите'): 'тире', ('потому', 'патаму'): 'трампу', ('что', 'шта'): 'штат', ('переждать', 'пе